In [1]:
from bs4 import BeautifulSoup
import requests 
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
import time
from selenium.webdriver.common.by import By
import pandas as pd
import tqdm
import pickle as pkl

## Minerador de URL's

In [2]:
## Escolher o site com o esporte selecionado:
# Mantenha o padrão de url para formatação de nome automática.
url = "https://www.slamonline.com/category/news/nba"

options = webdriver.ChromeOptions()

driver = webdriver.Chrome(options=options)

driver.set_window_size(1920, 1080)

driver.get(url)

time.sleep(1)

## Looping infinito para a coleta dos dados:
while True:

    driver.execute_script(f"window.scrollTo(0, document.body.scrollHeight)")

    time.sleep(1)

    ## Escolha um site com rolagem infinita com botão 'load more'.
    ## Definam a expressão re de acordo com a Biblioteca Selenium para clicar no botão Load More.
    driver.find_element(By.XPATH, '//button[normalize-space()="Load More"]').click()

    time.sleep(3)

    ## Ache os elementos que contenham as urls do site (<a></a>)
    links = driver.find_elements(By.CLASS_NAME, "blog-post-vert")

    ## Seleciona atributo 'href' da tag a de todos os links
    links_suit = [x.find_element(By.TAG_NAME, "a").get_attribute("href") for x in links]

    time.sleep(2)

    print(len(links_suit))

    ## Backup da lista de links
    with open(f"Links/Backup_{url.split('/')[-1]}.pkl", "wb") as f:
        pkl.dump(links_suit, f)

driver.close()

26


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=114.0.5735.198)
Stacktrace:
#0 0x557c4a2084e3 <unknown>
#1 0x557c49f37c76 <unknown>
#2 0x557c49f11c6c <unknown>
#3 0x557c49f97f8f <unknown>
#4 0x557c49faad66 <unknown>
#5 0x557c49f92de3 <unknown>
#6 0x557c49f682dd <unknown>
#7 0x557c49f6934e <unknown>
#8 0x557c4a1c83e4 <unknown>
#9 0x557c4a1cc3d7 <unknown>
#10 0x557c4a1d6b20 <unknown>
#11 0x557c4a1cd023 <unknown>
#12 0x557c4a19b1aa <unknown>
#13 0x557c4a1f16b8 <unknown>
#14 0x557c4a1f1847 <unknown>
#15 0x557c4a201243 <unknown>
#16 0x7f9d1ee94b43 <unknown>
